In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
from tqdm import trange
from src.svgd import SVGD
from src.gsvgd import FullGSVGDBatch
from src.kernel import RBF, BatchRBF
from src.manifold import Grassmann
import torch.autograd as autograd
from src.s_svgd import SlicedSVGD
from src.diffusion import Diffusion
from src.rand_mysvgd import mySVGD
import pickle
import argparse
import time
import numpyro
from numpyro.infer import MCMC, NUTS
import numpyro.distributions as npr_dist
import jax.random as random
import jax.numpy as jnp
import jax
from src.Tmy_svgd import etmySVGD

/home/zhoujk/anaconda3/envs/AG/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
parser = argparse.ArgumentParser(description="Running xshaped experiment.")
parser.add_argument("--dim", type=int, default=50, help="dimension")
parser.add_argument("--effdim", type=int, default=5, help="dimension")
parser.add_argument("--lr", type=float, default=0.0001, help="learning rate")
parser.add_argument("--lr_g", type=float, default=0.0001, help="learning rate for S-SVGD")
parser.add_argument(
    "--delta", type=float, default=0.1, help="stepsize for projections"
)
parser.add_argument(
    "--T", type=float, default=1e-4, help="noise multiplier for projections"
)
parser.add_argument("--nparticles", type=int, default=2500, help="no. of particles")
parser.add_argument("--epochs", type=int, default=20000, help="no. of epochs")
parser.add_argument("--noise", type=str, default="True", help="whether to add noise")
parser.add_argument("--kernel", type=str, default="rbf", help="kernel")
parser.add_argument("--gpu", type=int, default=3, help="gpu")
parser.add_argument("--seed", type=int, default=2, help="random seed")
parser.add_argument("--suffix", type=str, default="", help="suffix for res folder")
parser.add_argument("--method", type=str, default="svgd", help="svgd, gsvgd or s-svgd")
parser.add_argument("--save_every", type=int, default=100, help="batch size")

args = parser.parse_args([])

In [3]:
dim = args.dim
lr = args.lr
lr_g = args.lr_g
delta = args.delta
T = args.T
nparticles = args.nparticles
epochs = args.epochs
seed = args.seed
eff_dims = 10
add_noise = True if args.noise == "True" else False
save_every = args.save_every  # save metric values every 100 epochs
print(f"Running for lr: {lr}, nparticles: {nparticles}")

device = torch.device(f"cuda:{args.gpu}" if args.gpu != -1 else "cpu")


Running for lr: 0.0001, nparticles: 2500


In [4]:
results_folder = f"./res/diffusion{args.suffix}/{args.kernel}_epoch{epochs}_lr{lr}_delta{delta}_n{nparticles}_dim{dim}"
results_folder = f"{results_folder}/seed{seed}"

In [5]:
results_folder

'./res/diffusion/rbf_epoch20000_lr0.0001_delta0.1_n2500_dim50/seed2'

In [6]:
torch.__version__

'1.13.0'

In [7]:
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

if args.kernel == "rbf":
    Kernel = RBF
    BatchKernel = BatchRBF

In [8]:
print(f"Device: {device}")
torch.manual_seed(seed)
np.random.seed(seed)

## initialize conditional diffusion
beta = 5
sigma = 0.1

distribution = Diffusion(dim, beta, device=device)

loc = torch.arange(0, dim+1, 1)[1:]
distribution.loc = loc
noise_covariance = torch.diag(sigma**2 * torch.ones(len(loc), device=device))
distribution.set_noise_covariance(noise_covariance)
distribution.set_C()

x_true = distribution.brownian_motion((1, dim))
u_true = distribution.solution(x_true)

obs_noise = torch.normal(0, sigma, size=(1, len(loc))).to(device)
obs = u_true[:, loc] + obs_noise
distribution.obs = obs

# initialize particles
x0 = distribution.brownian_motion((nparticles, dim))

C = distribution.C.cpu().numpy()
dt = distribution.dt.cpu().numpy()
loc = distribution.loc.cpu().numpy()
beta = distribution.beta




Device: cuda:3


In [9]:
## SVGD
print("Running SVGD")
x = x0.clone().requires_grad_()
# sample from variational density
kernel = Kernel(method="med_heuristic")

Running SVGD


In [10]:
## SVGD
print("Running SVGD")
x = x0.clone().requires_grad_()
# sample from variational density
kernel = Kernel(method="med_heuristic")
svgd = SVGD(distribution, kernel, optim.Adam([x], lr=lr), device=device)

start = time.time()
svgd.fit(x, epochs=20000, save_every=save_every)
elapsed_time = time.time() - start

fitted_method = svgd
#particles = fitted_method.particles
particles = x

method_name = "svgd"


print(x)
## save results
pickle.dump(
     {
            **{"x_true": x_true},
            **{"u_true": u_true},
            **{"particles": particles},
           **{"time": elapsed_time}
    },
    open(results_folder + f"/particles_svgd1.p", "wb")
)

# target distribution
torch.save(distribution, results_folder + '/target_dist.p')
print(x.shape)

Running SVGD


100%|██████████| 20000/20000 [12:09<00:00, 27.42it/s]

tensor([[-0.0484, -0.0274, -0.1041,  ..., -0.2649, -0.2524, -0.1976],
        [-0.0739, -0.0474, -0.1151,  ...,  0.1220,  0.2054,  0.1763],
        [-0.3630, -0.3516, -0.3474,  ...,  0.8955,  0.9117,  0.7918],
        ...,
        [-0.0681,  0.0053, -0.0250,  ..., -0.1095, -0.1279, -0.0886],
        [-0.0660, -0.0190, -0.0514,  ..., -0.2198, -0.0550, -0.1299],
        [-0.1246, -0.1137, -0.1550,  ...,  0.1209,  0.1719,  0.1737]],
       device='cuda:3', requires_grad=True)
torch.Size([2500, 50])


In [11]:
print("Running min_mySVGD")


x = x0.clone().requires_grad_()
# sample from variational density

lr = 0.001
theta = etmySVGD(kernel,device,distribution,  k = 2).update(x0,   n_iter = 20000,   lr= lr)
       

particles = theta

elapsed_time =0 
## save results
pickle.dump(
        {
            **{"x_true": x_true},
            **{"u_true": u_true},
            **{"particles": particles},
            **{"time": elapsed_time}
        },
open(results_folder + f"/particles_AUmp_svgd_2_test3.p", "wb")
)

print(particles)

Running min_mySVGD


100%|██████████| 20000/20000 [1:33:29<00:00,  3.57it/s]

tensor([[-0.0786, -0.0450, -0.2621,  ..., -0.3847, -0.1322, -0.0451],
        [-0.0110,  0.1066,  0.0248,  ..., -0.2259,  0.0065,  0.0650],
        [-0.1137, -0.0257, -0.1004,  ..., -0.0234,  0.0562,  0.1070],
        ...,
        [-0.0624, -0.0858, -0.0348,  ..., -0.1806, -0.1323, -0.1273],
        [-0.0194,  0.1067,  0.0101,  ..., -0.3722, -0.2851, -0.2092],
        [-0.0978, -0.1153, -0.1604,  ..., -0.0918, -0.0114,  0.0280]],
       device='cuda:3')


In [12]:
print("Running min_mySVGD")


x = x0.clone().requires_grad_()
# sample from variational density

lr = 0.001
theta = etmySVGD(kernel,device,distribution,  k = 5).update(x0,   n_iter = 20000,   lr= lr)
       

particles = theta

elapsed_time =0 
## save results
pickle.dump(
        {
            **{"x_true": x_true},
            **{"u_true": u_true},
            **{"particles": particles},
            **{"time": elapsed_time}
        },
open(results_folder + f"/particles_AUmp_svgd_5_test3.p", "wb")
)

print(particles)


Running min_mySVGD


100%|██████████| 20000/20000 [1:32:41<00:00,  3.60it/s]

tensor([[-0.0833, -0.0465, -0.2434,  ..., -0.3307, -0.1971, -0.0585],
        [-0.0022,  0.1132,  0.0611,  ..., -0.2780, -0.0420,  0.0516],
        [-0.0314, -0.0040, -0.0454,  ...,  0.0079,  0.0464,  0.1003],
        ...,
        [-0.0627, -0.0841, -0.0366,  ..., -0.2511, -0.1439, -0.1408],
        [ 0.0291,  0.0679,  0.0041,  ..., -0.3703, -0.3182, -0.2092],
        [-0.0977, -0.1139, -0.1548,  ..., -0.1591, -0.0341,  0.0146]],
       device='cuda:3')


In [13]:
print("Running min_mySVGD")


x = x0.clone().requires_grad_()
# sample from variational density

lr = 0.001
theta = etmySVGD(kernel,device,distribution,  k = 10).update(x0,  n_iter = 20000,   lr= lr)
       

particles = theta

## save results
pickle.dump(
        {
            **{"x_true": x_true},
            **{"u_true": u_true},
            **{"particles": particles},
            **{"time": elapsed_time}
        },
open(results_folder + f"/particles_AUmp_svgd_10_test3.p", "wb")
)

print(particles)

Running min_mySVGD


100%|██████████| 20000/20000 [1:32:03<00:00,  3.62it/s]

tensor([[-0.0861, -0.0490, -0.2318,  ..., -0.3796, -0.2823, -0.0552],
        [ 0.0255,  0.0682,  0.0022,  ..., -0.2502, -0.0336,  0.0549],
        [ 0.0115, -0.0527, -0.1295,  ..., -0.0350, -0.0472,  0.0969],
        ...,
        [-0.0660, -0.0828, -0.0399,  ..., -0.2397, -0.1170, -0.1341],
        [ 0.0260,  0.0552, -0.0105,  ..., -0.3377, -0.2979, -0.2024],
        [-0.1856, -0.1353, -0.1478,  ..., -0.0676,  0.0490,  0.0180]],
       device='cuda:3')


In [14]:
## S-SVGD
# sample from variational density
print("Running S-SVGD")
x_s_svgd = x0.clone().requires_grad_()
s_svgd = SlicedSVGD(distribution, device=device)

start = time.time()
x_s_svgd, metric_s_svgd = s_svgd.fit(
    samples=x_s_svgd, 
    n_epoch=epochs, 
    lr=lr_g,
    eps=lr,
    save_every=save_every
)
elapsed_time = time.time() - start

fitted_method = s_svgd
particles = x_s_svgd

pickle.dump(
{
            **{"x_true": x_true},
            **{"u_true": u_true},
            **{"particles": particles},
            **{"time": elapsed_time}
        },
open(results_folder + f"/particles_ssvgd_test3.p", "wb")
)

print(particles)

Running S-SVGD


100%|██████████| 20000/20000 [2:15:40<00:00,  2.46it/s]  

tensor([[-0.0642, -0.0625, -0.3671,  ..., -0.3944, -0.2932, -0.0645],
        [ 0.0601,  0.0927,  0.0366,  ..., -0.2696, -0.0451,  0.0337],
        [ 0.0533, -0.0822, -0.1775,  ..., -0.0572, -0.0545,  0.0883],
        ...,
        [-0.0646, -0.1176, -0.0204,  ..., -0.2579, -0.1275, -0.1460],
        [ 0.0549,  0.0809,  0.0119,  ..., -0.3554, -0.3223, -0.2090],
        [-0.2589, -0.1724, -0.1759,  ..., -0.0706,  0.0318,  0.0094]],
       device='cuda:3', grad_fn=<CloneBackward0>)
